In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime
import time

In [2]:
def check_symbol (stock_df ,label) :
    test_dr = pd.DataFrame()
    for i in range (len(stock_df)):
        if len(stock_df.iloc[i][label]) == 4:
            test_dr = test_dr.append(stock_df[i:i+1]).reset_index(drop=True)
    return test_dr

In [3]:
def extract(Stock_list, whole_df , label):
    start = 0
    end = 0
    point = Stock_list[0]
    for i in range (len(whole_df)):
        if whole_df.iloc[i][label] >= str(point):
            #print (whole_df.iloc[i][label] , i)
            if start != 0 :
                end = i
                break
            point = Stock_list[len(Stock_list)-1]
            start = i
    test =  pd.DataFrame(whole_df[start:end+1])
    return test

In [4]:
def find_index(Stock_list, extract, label,  null) :
    institutional = pd.DataFrame()
    for i in range (len(Stock_list)):
        a = extract[extract[label] == str(Stock_list[i])].index
        if len(a)== 1  :
            institutional=institutional.append(extract.iloc[a])
        elif len(a) == 0 :
            institutional=institutional.append(null)
    return institutional.reset_index(drop=True)

In [5]:
def devide(list1 , list2) :
    out_come = list()
    for i in range (len(list1)):
        if type (list1[i]) != float and  int(list2[i].replace(",", "")) >0 :
            x = format(int(list1[i].replace(",", "")) / int(list2[i].replace(",", "")) ,'.3f')
        else :
            x = 0
        out_come.append(x)
    return out_come

In [6]:
Stock_list = pd.read_csv('name.csv')["name"]
date_list = pd.read_csv('date.csv')['date'].tolist()
whole_data = pd.DataFrame()
Stock_index = pd.read_csv('Market_index.csv')

i=0
today = datetime.date.today()
day = str (today-datetime.timedelta(days=i+553)).replace("-", "")
day

In [7]:
# generate date
i = 0
today = datetime.date.today()
day = str (today-datetime.timedelta(days=i)).replace("-", "")
record_day =  max(date_list) 
sleep_download_time = 3

while int(day) > record_day  :
#while i < 4000  :
    # get basic data
    
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + day + '&type=ALL')
    if len(r.text) < 100 :
        print (day ," is not work day!")
        time.sleep(sleep_download_time)
        i+=1
        day = str (today-datetime.timedelta(days=i)).replace("-", "")
        continue
        
    label_ = "證券代號"
    df = pd.read_csv(StringIO(r.text.replace("=", "")),  header=["證券代號" in l for l in r.text.split("\n")].index(True)-1).drop(['Unnamed: 16'],axis = 1)
    df = df.dropna().reset_index(drop=True)
    time.sleep(sleep_download_time)
    
    # process the basic data by name list
    base_df = extract(Stock_list , df , label_).reset_index(drop=True)
    stock_df = find_index(Stock_list, base_df, label_,  df[-1:])
    
    ## get institution data
    r = requests.get('https://www.twse.com.tw/fund/T86?response=csv&date=' + day + '&selectType=ALL')
    r = r.text.replace("=", "")
    df2 = pd.read_csv(StringIO(r.replace("  \n", "")),  header=["證券代號" in l for l in r.split("\r\n")].index(True))
    df2.sort_values(by='證券代號' , ascending= True, axis =0 , inplace=True)
    df2 = df2.reset_index(drop=True).drop(['Unnamed: 19'],axis = 1)
    
    institutional = extract(Stock_list , df2 , label_).reset_index(drop=True)
    institutional = find_index(Stock_list, institutional, label_,  df2[-1:])
    time.sleep(sleep_download_time)
    ## process margin data
    r = requests.get('https://www.twse.com.tw/exchangeReport/MI_MARGN?response=csv&date=' + day + '&selectType=ALL')
    r = r.text.replace("=", "")
    df3 = pd.read_csv(StringIO(r), header=["股票代號" in l for l in r.split("\n")].index(True)).drop(['Unnamed: 16'],axis = 1)

    label_ = "股票代號"
    margin = extract(Stock_list, df3 , label_).reset_index(drop=True)
    margin = find_index(Stock_list, margin, label_,  df3[-1:])
    ratio = devide(margin['今日餘額.1'], margin['今日餘額'] )
    margin ['ratio'] = ratio 
    time.sleep(sleep_download_time)
    
    # merge to basic df
    stock_df['融資買'] = margin['買進']
    stock_df['融資賣'] = margin['賣出']
    stock_df['融資餘'] = margin['今日餘額']
    stock_df['融卷買'] = margin['買進.1']
    stock_df['融卷賣'] = margin['賣出.1']
    stock_df['融卷餘'] = margin['今日餘額.1']
    stock_df['ratio'] = margin ['ratio']

    stock_df['外資進出'] = institutional['外陸資買賣超股數(不含外資自營商)']
    stock_df['外資進'] = institutional['外陸資買進股數(不含外資自營商)']
    stock_df['外資出'] = institutional['外陸資賣出股數(不含外資自營商)']
    stock_df['投信進出'] = institutional['投信買賣超股數']
    stock_df['投信進'] = institutional['投信買進股數']
    stock_df['投信出'] = institutional['投信賣出股數']
    stock_df['自營進出'] = institutional['自營商買賣超股數(自行買賣)']
    stock_df['自營進'] = institutional['自營商買進股數(自行買賣)']
    stock_df['自營出'] = institutional['自營商賣出股數(自行買賣)']
    stock_df['避險進出'] = institutional['自營商買賣超股數(避險)']
    stock_df['避險進'] = institutional['自營商買進股數(避險)']
    stock_df['避險出'] = institutional['自營商賣出股數(避險)']
    stock_df['三大'] = institutional['三大法人買賣超股數']
    
    #stock_df  = stock_df.dropna().reset_index(drop=True)
    
    ## add date label
    date = [day for i in range(len(stock_df))]
    stock_df ["date"] = date
    whole_data = whole_data.append(stock_df)
    date_list.append(int(day))
    
    # for market index
    index_set = ['發行量加權股價指數' , '未含電子指數' , '電子類兩倍槓桿指數' ]
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+ day +'&type=IND')
    df = pd.read_csv(StringIO(r.text.replace("價格指數(臺灣證券交易所)\"\r\n", "")),  header=["指數" in l for l in r.text.split("\r\n")].index(True)).drop(['漲跌(+/-)','特殊處理註記','Unnamed: 6'],axis = 1)
    df1 = df.dropna()
    df1.columns = ['Index','收盤','漲跌','漲跌%']
    index = pd.DataFrame()
    for x in index_set :
        index = index.append(df1[df1.Index == x])
    index['date'] = [int(day) for x in range (len(index))]
    Stock_index = Stock_index.append(index)
    time.sleep(sleep_download_time)    
    print (day)
    
    i+=1
    day = str (today-datetime.timedelta(days=i)).replace("-", "")

20210422  is not work day!
20210421


In [8]:
date = pd.DataFrame()
date['date'] = date_list
date = date.drop_duplicates()
date = date.sort_values(by= 'date',ascending=False)
date.to_csv('date.csv', index = False)

In [9]:
date

,date
529,20210421
0,20210420
1,20210419
2,20210416
3,20210415
...,...
524,20190221
525,20190220
526,20190219
527,20190218


In [10]:
drop = whole_data.dropna().reset_index(drop=True)
whole_data = drop.drop(['漲跌(+/-)', '證券名稱' ,'最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量'],axis = 1)

In [11]:
for l in whole_data.columns : 
    list_ = []
    for i in range(len(whole_data)) :
        k = whole_data[l][i]
        if type(k) == str :
            try :
                list_.append(float (k.replace(",", "")))
            except:
                print(l , i , k)
                list_.append(0)
                continue
        else : 
            list_.append(float(k))
    whole_data[l] = list_

In [12]:
Stock_index = Stock_index.drop_duplicates()
Stock_index = Stock_index.reset_index(drop=True)
Stock_index

,Index,收盤,漲跌,漲跌%,date
0,發行量加權股價指數,16211.7,264.85,1.66,20210303
1,未含電子指數,16115,273.49,1.73,20210303
2,電子類兩倍槓桿指數,40860.8,1287.77,3.25,20210303
3,發行量加權股價指數,15946.9,6.92,-0.04,20210302
4,未含電子指數,15841.5,60.53,-0.38,20210302
...,...,...,...,...,...
1535,未含電子指數,18732.2,20.89,0.11,20210420
1536,電子類兩倍槓桿指數,42467.7,411.4,0.98,20210420
1537,發行量加權股價指數,"17,202.11",121.76,-0.70,20210421
1538,未含電子指數,"18,738.99",6.76,0.04,20210421


In [13]:
for l in ['收盤','漲跌','漲跌%'] : 
    list_ = []
    for i in range(len(Stock_index)) :
        k = Stock_index[l][i]
        if type(k) == str :
            try :
                list_.append(float (k.replace(",", "")))
            except:
                print(l , i , k)
                list_.append(0)
                continue
        else : 
            list_.append(float(k))
    Stock_index[l] = list_

In [14]:
whole_data = whole_data [whole_data['開盤價'] > 0]
whole_data

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,31447338.0,12669.0,1.625491e+09,50.70,53.00,50.30,52.20,1.50,11.92,...,287000.0,0.0,989197.0,1572197.0,583000.0,39000.0,78000.0,39000.0,-577943.0,20210421.0
1,1103.0,16272571.0,5791.0,4.077864e+08,23.50,25.95,23.50,25.60,1.00,9.34,...,12298.0,0.0,-35000.0,41000.0,76000.0,0.0,0.0,0.0,-630702.0,20210421.0
2,1104.0,11669599.0,4991.0,3.089068e+08,26.40,27.30,25.60,27.10,0.65,14.19,...,0.0,60000.0,-7000.0,0.0,7000.0,0.0,0.0,0.0,630000.0,20210421.0
3,1108.0,8585635.0,3355.0,1.505281e+08,17.35,18.00,16.85,17.70,0.35,18.25,...,0.0,0.0,-3000.0,0.0,3000.0,0.0,0.0,0.0,637000.0,20210421.0
4,1109.0,2298671.0,1173.0,6.580208e+07,28.45,29.20,27.80,28.85,0.75,9.81,...,0.0,0.0,-919.0,82.0,1001.0,0.0,0.0,0.0,541081.0,20210421.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,9944.0,213656.0,215.0,4.381427e+06,20.55,20.55,20.45,20.45,0.10,29.64,...,0.0,0.0,-1000.0,0.0,1000.0,0.0,0.0,0.0,-1000.0,20210421.0
905,9945.0,7756908.0,4533.0,4.164597e+08,52.90,54.20,52.60,54.10,1.00,9.47,...,26092.0,0.0,-53000.0,0.0,53000.0,177000.0,182000.0,5000.0,1735725.0,20210421.0
906,9946.0,1104206.0,614.0,1.896844e+07,17.25,17.50,16.95,17.15,0.05,30.09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-273697.0,20210421.0
907,9955.0,297697.0,265.0,5.567553e+06,18.80,18.80,18.50,18.65,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83000.0,20210421.0


In [15]:
previous = pd.read_csv('clear.csv')
previous = previous.drop_duplicates()
previous 

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,25377783.0,11335.0,1.297775e+09,51.10,52.80,50.20,50.70,0.70,11.58,...,0.0,6000.0,-122640.0,242000.0,364640.0,38000.0,130000.0,92000.0,-3529357.0,20210420.0
1,1103.0,13917605.0,5123.0,3.444128e+08,25.05,25.90,23.95,24.60,0.10,8.98,...,141515.0,0.0,7000.0,7000.0,0.0,0.0,0.0,0.0,-763485.0,20210420.0
2,1104.0,12292291.0,5348.0,3.306401e+08,27.50,28.45,25.80,26.45,0.25,13.85,...,0.0,0.0,5000.0,5000.0,0.0,0.0,0.0,0.0,-170000.0,20210420.0
3,1108.0,17873955.0,6406.0,3.133867e+08,17.00,18.30,17.00,17.35,0.25,17.89,...,0.0,0.0,2000.0,2000.0,0.0,0.0,0.0,0.0,-424220.0,20210420.0
4,1109.0,3422123.0,1716.0,9.790249e+07,28.90,29.50,28.05,28.10,0.80,9.56,...,0.0,0.0,848.0,1001.0,153.0,0.0,0.0,0.0,-6152.0,20210420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450608,9943.0,190438.0,168.0,1.281202e+07,67.50,67.50,67.20,67.20,0.20,11.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52000.0,20190920.0
450609,9945.0,2752245.0,1096.0,1.163660e+08,42.60,42.60,42.10,42.10,0.50,4.38,...,0.0,81000.0,40000.0,40000.0,0.0,-56000.0,0.0,56000.0,-684265.0,20190920.0
450610,9946.0,1184358.0,469.0,2.778130e+07,23.50,23.60,23.30,23.60,0.10,5.58,...,0.0,0.0,-78000.0,12000.0,90000.0,0.0,0.0,0.0,107000.0,20190920.0
450611,9955.0,538156.0,334.0,1.028029e+07,18.80,19.20,18.80,19.15,0.35,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44000.0,20190920.0


In [16]:
K = whole_data.append(previous).reset_index(drop=True)
K

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,31447338.0,12669.0,1.625491e+09,50.70,53.00,50.30,52.20,1.50,11.92,...,287000.0,0.0,989197.0,1572197.0,583000.0,39000.0,78000.0,39000.0,-577943.0,20210421.0
1,1103.0,16272571.0,5791.0,4.077864e+08,23.50,25.95,23.50,25.60,1.00,9.34,...,12298.0,0.0,-35000.0,41000.0,76000.0,0.0,0.0,0.0,-630702.0,20210421.0
2,1104.0,11669599.0,4991.0,3.089068e+08,26.40,27.30,25.60,27.10,0.65,14.19,...,0.0,60000.0,-7000.0,0.0,7000.0,0.0,0.0,0.0,630000.0,20210421.0
3,1108.0,8585635.0,3355.0,1.505281e+08,17.35,18.00,16.85,17.70,0.35,18.25,...,0.0,0.0,-3000.0,0.0,3000.0,0.0,0.0,0.0,637000.0,20210421.0
4,1109.0,2298671.0,1173.0,6.580208e+07,28.45,29.20,27.80,28.85,0.75,9.81,...,0.0,0.0,-919.0,82.0,1001.0,0.0,0.0,0.0,541081.0,20210421.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451517,9943.0,190438.0,168.0,1.281202e+07,67.50,67.50,67.20,67.20,0.20,11.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52000.0,20190920.0
451518,9945.0,2752245.0,1096.0,1.163660e+08,42.60,42.60,42.10,42.10,0.50,4.38,...,0.0,81000.0,40000.0,40000.0,0.0,-56000.0,0.0,56000.0,-684265.0,20190920.0
451519,9946.0,1184358.0,469.0,2.778130e+07,23.50,23.60,23.30,23.60,0.10,5.58,...,0.0,0.0,-78000.0,12000.0,90000.0,0.0,0.0,0.0,107000.0,20190920.0
451520,9955.0,538156.0,334.0,1.028029e+07,18.80,19.20,18.80,19.15,0.35,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44000.0,20190920.0


In [17]:
K.columns

Index(['證券代號', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差',
       '本益比', '融資買', '融資賣', '融資餘', '融卷買', '融卷賣', '融卷餘', 'ratio', '外資進出', '外資進',
       '外資出', '投信進出', '投信進', '投信出', '自營進出', '自營進', '自營出', '避險進出', '避險進', '避險出',
       '三大', 'date'],
      dtype='object')

In [18]:
K.to_csv('clear.csv', index = False)
Stock_index.to_csv('Market_index.csv', index = False)